##Imports

In [ ]:
import random
from google.colab import drive
import os
import pandas as pd
import requests
import sklearn.model_selection as sk
import time
import gspread
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build

##Parametros

In [ ]:
folder_text = "Adaptacion"
file_text = "experimento_adaptacion_"
folder_root = "/content/drive/My Drive/"
nombre_csv_links = "links.csv"
cantidad_de_experimentos = 12
number_of_packs = 1
number_of_superposition = 1
cantidad_de_textos = 2
seed = 50

folder_name = 'prueba deploy'

drive.mount('/content/drive', force_remount=False)

experimento ={0: range(6), 1: range(6, 12)}
formulario_para_experimento = 0

Mounted at /content/drive


##Cargar Textos

In [ ]:
# Check if the zip file is present and has been unzipped
if not os.path.exists("cefr-levelled-english-texts.zip"):
    # Download the dataset if the zip file is not present
    !kaggle datasets download -d amontgomerie/cefr-levelled-english-texts

if not os.path.exists("cefr_leveled_texts.csv"):  # Adjust this to match the folder name after unzipping
    # Unzip the file if the unzipped folder does not exist
    !unzip cefr-levelled-english-texts.zip

# Load the CSV file into a DataFrame
df = pd.read_csv('cefr_leveled_texts.csv')

# Get the minimum number of samples in any class
min_samples = df['label'].value_counts().min()

# Downsample each class to have the same number of samples as the smallest class
df_balanced = df.groupby('label').apply(lambda x: x.sample(n=min_samples, random_state=60)).reset_index(drop=True)

distribution = df_balanced['label'].value_counts()
train, div = sk.train_test_split(df_balanced, test_size=0.2, random_state=70)
dev, holdout = sk.train_test_split(div, test_size=0.5, random_state=50)

train = train.reset_index(drop=True)
div = div.reset_index(drop=True)
holdout = holdout.reset_index(drop=True)

true_list = pd.concat([dev, train])
text_org = true_list['text'].tolist()
labels = true_list['label'].tolist()

Dataset URL: https://www.kaggle.com/datasets/amontgomerie/cefr-levelled-english-texts
License(s): CC0-1.0
  0% 0.00/1.36M [00:00<?, ?B/s]
100% 1.36M/1.36M [00:00<00:00, 72.3MB/s]
Archive:  cefr-levelled-english-texts.zip
  inflating: cefr_leveled_texts.csv  


<ipython-input-5-2a30633cbf7a>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby('label').apply(lambda x: x.sample(n=min_samples, random_state=60)).reset_index(drop=True)


In [ ]:
# Define the path for the new folder
folder_path = folder_root + folder_text

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    print(f"Folder at {folder_path} dont exist")

text_adaptation = []
for i in range (0, cantidad_de_experimentos):
  file_name= file_text + f"{i}.csv"
  csv_file_path = folder_path + '/' + file_name
  df = pd.read_csv(csv_file_path)
  labels_list = df['Adaptacion'].tolist()
  text_adaptation.append(labels_list)

##Crear Paquetes de indices

In [ ]:
random.seed(seed)
n = len(text_org)
random_indices = random.sample(range(n), cantidad_de_textos)

In [ ]:
def create_balanced_index_packs(indices, N, M):
    random.shuffle(indices)
    # Initialize empty packs and track their sizes
    packs = [[] for _ in range(N)]
    pack_sizes = [0] * N  # Track the number of texts in each pack

    for indice in indices:
        indexes = []
        # Find the average size of the packs
        avg_pack_size = sum(pack_sizes) / N

        # Select M unique packs that have sizes below or close to the average
        while len(indexes) < M:
            index = random.randint(0, N - 1)
            # Only add this index if it's not already chosen and if the pack is balanced
            if index not in indexes and pack_sizes[index] <= avg_pack_size:
                indexes.append(index)

        # Distribute the text to the selected packs
        for index in indexes:
            packs[index].append(indice)
            pack_sizes[index] += 1  # Update the pack size

    return packs


index_packs = create_balanced_index_packs(random_indices, number_of_packs, number_of_superposition)

for idx, pack in enumerate(index_packs):
    print(f"Pack {idx + 1}: {pack}")

Pack 1: [1018, 545]


##Crear paquetes de textos

In [ ]:
#diccionario de numeros con clases A1, A2, ..
cefr_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}

numb_to_class = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}

In [ ]:
def generar_lista_de_paquetes (matriz_de_indices):
  lista_de_paquetes_de_textos = []
  for lista_de_indices in matriz_de_indices:
    paquetes_de_textos = generar_paquete(lista_de_indices)
    lista_de_paquetes_de_textos.append(paquetes_de_textos)
  return lista_de_paquetes_de_textos

def generar_paquete(lista_de_indices):
  paquetes_de_textos = []
  for indice in lista_de_indices:
    lista_textos_adaptados = generar_lista_textos_adaptados(indice)
    texto_original = text_org[indice]
    paquetes_de_textos.append([texto_original, lista_textos_adaptados])
  return paquetes_de_textos

def generar_lista_textos_adaptados (indice):



  ##
  ##
  ##Modificar para que se adapte a si se esta usando experimento 0 o 1
  ##
  ##
  lista_de_textos_adaptados = []

  original_level = labels[indice]
  original_index  = cefr_mapping[original_level]
  valid_indices = range(max(0, original_index - 2), min(6, original_index + 3))

  for i in experimento[formulario_para_experimento]:
    if i in valid_indices:
      lista_de_textos_adaptados.append(text_adaptation[i][indice])
  return lista_de_textos_adaptados

In [ ]:
text_packs = generar_lista_de_paquetes(index_packs)

##Crear o abrir carpeta en drive

In [ ]:
# Define the path for the new folder
folder_path = folder_root + folder_name

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder created at {folder_path}")
else:
    print(f"Folder already exists at {folder_path}")

Folder already exists at /content/drive/My Drive/prueba deploy


##Plantillas para el formulario

In [ ]:
#dunciones para agregar los diferentes items
def add_secction(title, desc=''):
  tipo.append(0)
  titulo.append(title)
  descripcion.append(desc)
  valores.append('')
  obligatorio.append('')

def add_title(title, desc=''):
  tipo.append(1)
  titulo.append(title)
  descripcion.append(desc)
  valores.append('')
  obligatorio.append('')

def add_choice(title, val, oblig, desc=''):
  tipo.append(2)
  titulo.append(title)
  descripcion.append(desc)
  valores.append(val)
  obligatorio.append(oblig)

def add_paragraph(title, desc='', oblig=False):
  tipo.append(3)
  titulo.append(title)
  descripcion.append(desc)
  valores.append('')
  obligatorio.append(oblig)

In [ ]:
#Plantilla 1
#se comunican atravez de las listas
def plantilla_1 (j, length, text):
  titulo_seccion = f'Texto {j} de {length}'
  titulo_texto = 'Texto Original'
  titulo_clasificacion = 'Que clasificacion segun CEFR cree que es el texto'
  valores_clasificacion = 'A1, A2, B1, B2, C1, C2'

  add_secction(titulo_seccion)
  add_title(titulo_texto, text)
  add_choice(titulo_clasificacion, val=valores_clasificacion, oblig=True)

def plantilla_2 (j, length, criterios, org_text, adap_class, adap_text):
  titulo_seccion = f'Texto {j} de {length}, Adaptacion a {numb_to_class [adap_class]}'
  titulo_texto1 = 'Texto Original'
  titulo_texto2 = f'Adaptacion a {numb_to_class[adap_class]}'
  titulo_choice1 = 'El texto es coherente'
  valores_choice1 = 'Si, No'
  titulo_choice2 = 'Contiene la misma informacion que el texto original'
  valores_choice2 = 'Si, No'
  titulo_choice3 = 'Contiene vocabulario apto para el nivel adaptado'
  valores_choice3 = 'Si, No'
  titulo_choice4 = 'Las estructuras grematicales son aptas para el nivel adaptado'
  valores_choice4 = 'Si, No'
  titulo_choice5 = 'El texto es comprensible para el nivel al que fue adaptado'
  valores_choice5 = 'Si, No'
  titulo_choice6 = 'Teniendo en cuenta las respuestas anteriores, es una buena adaptacion del texto original al nivel requerido'
  valores_choice6 = 'Si, No'
  titulo_parrafo = 'Observaciones o sugerencias'

  add_secction(titulo_seccion, criterios)
  add_title(titulo_texto1, org_text)
  add_title(titulo_texto2, adap_text)
  add_choice(titulo_choice1, val=valores_choice1, oblig=True)
  add_choice(titulo_choice2, val=valores_choice2, oblig=True)
  add_choice(titulo_choice3, val=valores_choice3, oblig=True)
  add_choice(titulo_choice4, val=valores_choice4, oblig=True)
  add_choice(titulo_choice5, val=valores_choice5, oblig=True)
  add_choice(titulo_choice6, val=valores_choice6, oblig=True)
  add_paragraph(titulo_parrafo)

In [ ]:
def ejecutar_plantilla(informacion, plantilla):
  i = 0; #titulos
  j = 0; #textos
  for row in plantilla.itertuples():

    #agregar una seccion
    if row.Tipo_de_elemento == 'Seccion':
      if row.titulo_se_rellena_en_ejecucion == "Si" and row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_secction(informacion[0][i], informacion[1][j])
        i += 1
        j += 1
      elif row.titulo_se_rellena_en_ejecucion == "Si":
        add_secction(informacion[0][i], row.descripcion)
        i += 1
      elif row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_secction(row.titulo, informacion[1][j])
        j += 1
      else:
        add_secction(row.titulo, row.descripcion)

    #agregar un titulo
    elif row.Tipo_de_elemento == 'Titulo':
      if row.titulo_se_rellena_en_ejecucion == "Si" and row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_title(informacion[0][i], informacion[1][j])
        i += 1
        j += 1
      elif row.titulo_se_rellena_en_ejecucion == "Si":
        add_title(informacion[0][i], row.descripcion)
        i += 1
      elif row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_title(row.titulo, informacion[1][j])
        j += 1
      else:
        add_title(row.titulo, row.descripcion)

    #Agregar un multiple Choice
    elif row.Tipo_de_elemento == 'Multiple Choice':
      if row.titulo_se_rellena_en_ejecucion == "Si" and row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_choice(informacion[0][i], row.valores, row.es_obligatorio, informacion[1][j])
        i += 1
        j += 1
      elif row.titulo_se_rellena_en_ejecucion == "Si":
        add_choice(informacion[0][i], row.valores, row.es_obligatorio, row.descripcion)
        i += 1
      elif row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_choice(row.titulo, row.valores, row.es_obligatorio, informacion[1][j])
        j += 1
      else:
        add_choice(row.titulo, row.valores, row.es_obligatorio, row.descripcion)

    #Agregar un parrafo
    elif row.Tipo_de_elemento == 'Parrafo':
      if row.titulo_se_rellena_en_ejecucion == "Si" and row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_paragraph(informacion[0][i], informacion[1][j], row.es_obligatorio)
        i += 1
        j += 1
      elif row.titulo_se_rellena_en_ejecucion == "Si":
        add_paragraph(informacion[0][i], row.descripcion, row.es_obligatorio)
        i += 1
      elif row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_paragraph(row.titulo, informacion[1][j], row.es_obligatorio)
        j += 1
      else:
        add_paragraph(row.titulo, row.descripcion, row.es_obligatorio)


    else:
      print("Elemento no reconocido:", row.Tipo)


##buscar Plantilla

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the spreadsheet
url = 'https://docs.google.com/spreadsheets/d/1itQZXDTjLDCByitJ6Kvw6lXJMOGbVfIoSkByhAI09vY/edit?usp=sharing'
spreadsheet = gc.open_by_url(url)

# Loop through all worksheets
worksheets = spreadsheet.worksheets()
dfs = {}

for ws in worksheets:
    data = ws.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    dfs[ws.title] = df

plantillas = []
for sheet_name, plantilla in dfs.items():
  plantillas.append(plantilla)

##Crear CSV con elementos del formulario

In [ ]:
# folder_path = f'/content/drive/My Drive/{folder_name}'

# data_form = []

# criterios = "  criterio  "

# for i, pack in enumerate(text_packs):
#   file_name = 'CSV Paquete ' + str(i+1) + '.csv'

#   tipo = []
#   titulo = []
#   descripcion = []
#   valores = []
#   obligatorio = []

#   # Counter for the number of plantillas
#   plantilla_count = 0

#   #Armado de formulario
#   for j, lista_textos in enumerate(pack):
#     plantilla_1 (j, len(pack), lista_textos[0])
#     plantilla_count += 1  # Increment for plantilla_1

#     for k, adapt_text in enumerate(lista_textos[1]):
#       plantilla_2 (j + 1, len(pack), criterios, lista_textos[0], k, adapt_text)
#       plantilla_count += 1  # Increment for each plantilla_2


#   data = {
#       'Tipo': tipo,
#       'Titulo': titulo,
#       'Descripcion': descripcion,
#       'Valores': valores,
#       'Obligatorio': obligatorio
#   }

#   df = pd.DataFrame(data)
#   csv_file_path = folder_path + f'/{file_name}'
#   df.to_csv(csv_file_path, sep='¬', lineterminator='~', index=False)
#   Nombre_form = 'Formulario Paquete ' + str(i+1)
#   titulo_form = 'Paquete ' + str(i+1)
#   descripcion_form = ''
#   url = ''
#   datos = [file_name, Nombre_form, titulo_form, descripcion_form, url ,plantilla_count]
#   data_form.append(datos)
#   print(f"CSV file created at {csv_file_path}")

In [ ]:
folder_path = f'/content/drive/My Drive/{folder_name}'

data_form = []

criterios = "  criterio  "

paquetes = []

for i, pack in enumerate(text_packs):
  file_name = 'CSV Paquete ' + str(i+1) + '.csv'

  tipo = []
  titulo = []
  descripcion = []
  valores = []
  obligatorio = []

  cantidad_elementos = 0

  for j, lista_textos in enumerate(pack):
    textos = [lista_textos[0]]
    titulos = [f"Texto {j+1} de {len(pack)}"]
    informacion = [titulos, textos]
    ejecutar_plantilla (informacion, plantillas[0])
    cantidad_elementos += plantillas[0].shape[0]

    for clasificacion, adapt_text in enumerate(lista_textos[1]):
      textos = [lista_textos[0], adapt_text]
      titulos = [f"Texto {j+1} de {len(pack)}, Adaptacion a {numb_to_class[clasificacion]}",
                 f"Adaptacion a {numb_to_class[clasificacion]}"]
      informacion = [titulos, textos]
      ejecutar_plantilla (informacion, plantillas[1])
      cantidad_elementos += plantillas[1].shape[0]

  data = {
      'Tipo': tipo,
      'Titulo': titulo,
      'Descripcion': descripcion,
      'Valores': valores,
      'Obligatorio': obligatorio
  }

  df = pd.DataFrame(data)
  csv_file_path = folder_path + f'/{file_name}'
  df.to_csv(csv_file_path, sep='¬', lineterminator='~', index=False)
  Nombre_form = 'Formulario Paquete ' + str(i+1)
  titulo_form = 'Paquete ' + str(i+1)
  paquetes.append(titulo_form)
  descripcion_form = ''
  url = ''
  datos = [file_name, Nombre_form, titulo_form, descripcion_form, url , cantidad_elementos]
  data_form.append(datos)
  print(f"CSV file created at {csv_file_path}")

CSV file created at /content/drive/My Drive/prueba deploy/CSV Paquete 1.csv


##Espera para que se cargen los csv in drive

In [ ]:
time.sleep(15)

##crear formulario a traves de Apps Script

In [ ]:
def make_request(url, params):
    """Send a GET request to the Apps Script and process the response."""
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            try:
                result = response.json()  # Parse JSON response
                return result.get('url'), result.get('formId')
            except requests.exceptions.JSONDecodeError:
                print("Response is not valid JSON:", response.text)
                return None, None
        else:
            print(f"Failed to connect: {response.status_code}")
            return None, None
    except requests.RequestException as e:
        print(f"Error during request: {e}")
        return None, None

In [ ]:
# Base URL for the Apps Script
url = 'https://script.google.com/macros/s/AKfycbxrMMWc_s4g6fDOYbd3hPB7ygjH29wioBaVw7UkzMwRBaJ9A7BFeT4xMJTRZmwTUbe-/exec'

form_urls = []
form_ids = []

for data in data_form:
  data[4] = ''
  cantidad_elementos = data[5]
  for j in range ((cantidad_elementos//10) + 1):
    params = {
        'folderName': folder_name,
        'fileName': data[0],
        'formName': data[1],
        'title': data[2],
        'descipcion': data[3],
        'formId': data[4],
        'startElement': 10*j,
        'stopElement': 10*(j+1) - 1
    }

    form_url, form_id = make_request(url, params)
    data[4] = form_id
    if form_url:
        print("star element: ", 10*j, "stop element: ", 10*(j+1) - 1)
        print(f"Form URL: {form_url}, Form ID: {form_id}")
    else:
        print("Failed to retrieve form details.")

  form_urls.append(form_url)
  form_ids.append(form_id)

star element:  0 stop element:  9
Form URL: https://docs.google.com/forms/d/1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA/edit, Form ID: 1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA
star element:  10 stop element:  19
Form URL: https://docs.google.com/forms/d/1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA/edit, Form ID: 1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA
star element:  20 stop element:  29
Form URL: https://docs.google.com/forms/d/1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA/edit, Form ID: 1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA
star element:  30 stop element:  39
Form URL: https://docs.google.com/forms/d/1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA/edit, Form ID: 1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA
star element:  40 stop element:  49
Form URL: https://docs.google.com/forms/d/1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA/edit, Form ID: 1wbpB8m7HqflCK4zxceOGu1UhW7wQ0kfy6N3OzvcvQOA
star element:  50 stop element:  59
Form URL: https://docs.google.com/forms/d/1wbpB8m7H

##Crear csv con links

In [ ]:
data = {
      'Paquete': paquetes,
      'Links': form_urls,
      'Form ID': form_ids
  }

df = pd.DataFrame(data)
csv_file_path = folder_path + f'/{nombre_csv_links}'
df.to_csv(csv_file_path, index=False)
print(f"CSV file created at {csv_file_path}")

CSV file created at /content/drive/My Drive/prueba deploy/links.csv
